### FRB Search Volume

Given a DM limit of an FRB search, compute the search volume of the ALFABURST survey.

Dependency: NE2001

Adapted from code by Jayanth Chennamangalam and Evan Keane (calcfrbvol.py)

**Assumptions:**
1. All FRBs originate outside the Galaxy
2. There exists a constant electron density in the IGM (Ioka 2003 and Inoue 2004)
3. Distances are cosmological, $d(z) = \frac{c \cdot z}{H_0}$
4. The number density of galaxies is $n_{\textrm{gal}} = \frac{0.01}{\textrm{Mpc}^{3}}$
5. The host galaxy provides a DM of 100
6. Using a constant z to DM relation of $z = \frac{\textrm{DM}}{1200}$
7. FRBs are flat spectrum, standard candles

In [82]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units
from astropy.coordinates import SkyCoord

%matplotlib inline

In [96]:
# Parameters
dmMax = 10000. # Dispersion measure limit of search in cm^-3 pc
dmHost = 100. # Assume DM=100 component from source, this asusmption is that the source is embedded in a galaxy

cognizeLog = '../data/cognize.pkl'
gal_cut = 7. # Cut points +/- degrees around the galactic plane

nBeams = 7. # number of beams, for ALFA this is 7
beamFWHM = nBeams * (3.5 / 60.) # beam FWHM in degrees
areaDeg = np.pi * (beamFWHM/2.)**2. # beam area in deg^2

nGalaxies = 1e-2 # number density of galaxies, Mpc^-3

In [84]:
# Constants
deg2rad = np.pi / 180.
rad2deg = 180. / np.pi
Gpc32Mpc3 = 1e9 # convert Gpc^3 to Mpc^3

In [85]:
def coMovingDist(z):
    """Co-moving distance in Gpc, Eq. 3 from Lorimer et al. 2013
    z: float, redshift
    """
    c = 299792.458 # km s^-1
    H0 = 68. # km s^-1 Mpc^-1
    OmegaM = 0.32
    OmegaLambda = 0.68
    
    integrand = lambda zp: 1. / np.sqrt(OmegaM * ((1. + zp)**3.) + OmegaLambda)
    dd, err = scipy.integrate.quad(integrand, 0., z)
    return ((c/H0) * dd) * 1e-3

In [86]:
# Load pandas dataframe from cognize log to get (RA, Dec) and integration time
df = pd.read_pickle(cognizeLog)

In [92]:
activeObs = df[df.status==1] # select log of active observations

# NOTE:only using the central beam
# RA in hours, DEC in degrees and flipped
ra0 = activeObs['RA0'].values * (15. * np.pi / 180.) # Hours to radians
dec0 = (activeObs['DEC0'].values * (np.pi / 180.) ) # Degrees to radians
tint = activeObs['tint'].values # integration time in seconds

totalObsTime = np.sum(tint)
print 'Total observation time: %i seconds'%(totalObsTime)

# Convert (RA, Dec) to Galactic (lat, long)
coords = SkyCoord(ra=ra0 * astropy.units.rad, dec=dec0 * astropy.units.rad, frame='icrs')
gall = coords.galactic.l.degree
galb = coords.galactic.b.degree

# cut out a galactic plane strip
galCutObsTime = totalObsTime
if gal_cut is not None:
    idx = np.argwhere(np.abs(galb) > gal_cut)
    gall = gall[idx].flatten()
    galb = galb[idx].flatten()
    tint = tint[idx].flatten()

    galCutObsTime = np.sum(tint)
    print 'Galaxy Cut Observation time: %i seconds'%(galCutObsTime)

Total observation time: 3830640 seconds
Galaxy Cut Observation time: 2175300 seconds


In [88]:
# NE2001 Setup
NE2001root = "/local/griffin/projects/NE2001/" # HARDCODE
NE2001bin = NE2001root + "bin.NE2001/NE2001"
NE2001inp = NE2001root + "input.NE2001/"
tailProcs = "| tr -s ' ' | sed 's/^[ \t]*//'"
# Choose some large distance outside the Galaxy, so that NE2001 returns the
# DM for the entire line-of-sight within the Galaxy
distBound = 100. # kpc
dirDist2DM = "-1" # direction: distance to DM
dirDM2Dist = "1" # direction: DM to distance

In [90]:
# initialization
nGalSurvey = 0. # number of galaxies in the survey
volSurvey = 0. # volume of surveyed universe
areaSurvey = 0. # area of sky surveyed
zVals = [] # distance in z calculated from the NE2001 model based on the maximum DM search

#for l,b in zip(gall[:10],galb[:10]):
print 'Pointings:', gall.shape[0]
for cnt,(l,b) in enumerate(zip(gall,galb)):
    if cnt % 1000 == 0: print cnt
    # Calculate the galactic DM for each poiinting
    # build NE2001 command string
    cmd = " ".join(('cd %s;'%NE2001inp, NE2001, str(l), str(b), str(distBound), dirDist2DM, tailProcs))
    outputBuffer = sp.check_output(cmd, shell=True, universal_newlines=True, cwd=NE2001inp) # run NE2001 command
    cols = outputBuffer.split("\n")[7].split(" ") # get the 7th line
    dmGal = float(cols[0]) # galactic DM
    
    dmDiff = dmMax - (dmGal + dmHost) # > 0 if there is excess DM outside the host and our galaxy
    if dmDiff > 0.:
        zz = dmDiff / 1200. # calculate the distance in z to the source using the relation DM ~ 1200 z cm^-3 pc
                            # for z <= 2 (Lorimer et al. 2007), based on Ioka (2003) and Inoue (2004)
        zVals.append(zz)
        
        dd = coMovingDist(zz) # compute the Hubble distance, in Gpc
        
        # Volume, assuming a cone, V = pi r^2 h / 3, where r = dd theta / 2 (small angle approximation), and h = dd
        # this ignores any sensitivity beyond the beam FWHM
        vol = (np.pi * ((dd * (beamFWHM/2.))**2.) * dd) / 3.
        vol *= Gpc32Mpc3 # convert from Gpc^3 to Mpc^3
        nGal = vol * nGalaxies  # number of galaxies in volume
        
        nGalSurvey += nGal
        volSurvey += vol
        areaSurvey += areaDeg

zVals = np.array(zVals)
zMin = np.min(zVals)
zMax = np.max(zVals)
zMean = np.mean(zVals)

 Pointings: 36255
0
100
200
300
400
500
600
700
800
900
1000
1100
1200
1300
1400
1500
1600
1700
1800
1900
2000
2100
2200
2300
2400
2500
2600
2700
2800
2900
3000
3100
3200
3300
3400
3500
3600
3700
3800
3900
4000
4100
4200
4300
4400
4500
4600
4700
4800
4900
5000
5100
5200
5300
5400
5500
5600
5700
5800
5900
6000
6100
6200
6300
6400
6500
6600
6700
6800
6900
7000
7100
7200
7300
7400
7500
7600
7700
7800
7900
8000
8100
8200
8300
8400
8500
8600
8700
8800
8900
9000
9100
9200
9300
9400
9500
9600
9700
9800
9900
10000
10100
10200
10300
10400
10500
10600
10700
10800
10900
11000
11100
11200
11300
11400
11500
11600
11700
11800
11900
12000
12100
12200
12300
12400
12500
12600
12700
12800
12900
13000
13100
13200
13300
13400
13500
13600
13700
13800
13900
14000
14100
14200
14300
14400
14500
14600
14700
14800
14900
15000
15100
15200
15300
15400
15500
15600
15700
15800
15900
16000
16100
16200
16300
16400
16500
16600
16700
16800
16900
17000
17100
17200
17300
17400
17500
17600
17700
17800
17900
18000
18100
18

In [95]:
print "Total time observing = %i s"%(totalObsTime)
print "Observing time outside the galactic plane (+/-%.1f degrees) = %i s"%(gal_cut, galCutObsTime)
print "Total volume surveyed = %2.3e Gpc^3" % (volSurvey / Gpc32Mpc3), "= %2.3e Mpc^3" % volSurvey
print "Total area surveyed = %2.3e sq. deg." % areaSurvey
print "z (max = %2.3f, min = %2.3f, mean = %2.3f)" % (zMax, zMin, zMean)
print "Total number of galaxies (assuming n = %2.3f" % nGalaxies, "Mpc^-3) = %2.3f" % nGalSurvey

Total time observing = 3830640 s
Observing time outside the galactic plane (+/-7.0 degrees) = 2175300 s
Total volume surveyed = 1.177e+06 Gpc^3 = 1.177e+15 Mpc^3
Total area surveyed = 4.748e+03 sq. deg.
z (max = 8.234, min = 7.997, mean = 8.203)
Total number of galaxies (assuming n = 0.010 Mpc^-3) = 11765006312257.869
